In [1]:
!nvidia-smi

Tue May 30 23:34:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    Off  | 00000000:81:00.0 Off |                  N/A |
| 41%   31C    P8     9W / 280W |     67MiB / 24219MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Configurations

In [2]:
EXP_NAME = "pd-exp222"
ENV = "local"
DEBUG = False

In [3]:
class CFG:
    env = ENV
    exp_name = EXP_NAME
    debug = DEBUG
    input_dir = None
    output_dir = None
    print_freq = 50
    num_workers = 8

    # ====================================================
    # dataset
    # ====================================================
    competition_name = "tlvmc-parkinsons-freezing-gait-prediction"
    target_cols = ["StartHesitation", "Turn", "Walking"]
    seq_len = 5000
    shift = 2500
    offset = 1250

    # ====================================================
    # scheduler
    # ====================================================
    batch_size = 24
    epochs = 11
    num_warmup_steps = 10

    # ====================================================
    # optimizer
    # ====================================================
    lr = 1e-3
    weight_decay = 0.05  # default: 0.0

    # ====================================================
    # cross validation
    # ====================================================
    seed = 0
    n_fold = 5
    train_fold = [0, 1, 2, 3, 4]

    # ====================================================
    # mode settings
    # ====================================================
    train_by_fold = False
    train_all = True
    upload_kaggle_dataset = True
    inference = False

In [4]:
if CFG.debug:
    CFG.epochs = 2
    CFG.train_fold = [0, 1]

In [5]:
if CFG.env == "colab" and CFG.upload_kaggle_dataset:
    !pip install kaggle
    from google.colab import files, drive
    uploaded = files.upload()
    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])))
    # Then move kaggle.json into the folder where the API expects to find it.
    !mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

# Directory setting

In [6]:
from pathlib import Path


print(f"env is {CFG.env}.")
if CFG.env == "colab":
    # colab環境
    from google.colab import drive
    drive.mount("/content/drive")
    input_path = Path("./drive/MyDrive/Colab Notebooks/input")
    output_path = Path("./drive/MyDrive/Colab Notebooks/output")
    CFG.input_dir = input_path
    CFG.output_dir = output_path / CFG.exp_name
    output_path_tmp = Path("./drive/MyDrive/Colab\ Notebooks/output")
    CFG.output_dir_tmp = output_path_tmp / CFG.exp_name
    if not CFG.output_dir.exists():
        CFG.output_dir.mkdir()
    !pip install polars==0.17.12
    !pip install transformers==4.20.1
elif CFG.env == "local":
    # ローカルサーバ
    CFG.input_dir = Path("../mnt/input/")
    CFG.output_dir = Path("../mnt/output/") / CFG.exp_name
    if not CFG.output_dir.exists():
        CFG.output_dir.mkdir()
        print(f"Create dir: {CFG.output_dir}")
    !pip install polars==0.17.12
elif CFG.env == "kaggle":
    # kaggle環境
    CFG.input_dir = Path("../input/") / CFG.competition_name
    CFG.output_dir = Path("./")

env is local.


# Library

In [7]:
import gc
import os
import time
import math
import pathlib
import random
import warnings
import numpy as np
import pandas as pd
import polars as pl
from tqdm.auto import tqdm
from IPython.display import display
from sklearn.model_selection import train_test_split, StratifiedGroupKFold
from sklearn.metrics import accuracy_score, average_precision_score
from sklearn.preprocessing import StandardScaler, RobustScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import transformers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
print(f"transformers.__version__: {transformers.__version__}")

transformers.__version__: 4.20.1


In [8]:
warnings.filterwarnings("ignore")

In [9]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Utilities

In [10]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))

# Data Loading

In [11]:
tdcsfog_metadata = pd.read_csv(CFG.input_dir / "tdcsfog_metadata.csv")
defog_metadata = pd.read_csv(CFG.input_dir / "defog_metadata.csv")
subjects = pd.read_csv(CFG.input_dir / "subjects.csv")
events = pd.read_csv(CFG.input_dir / "events.csv")
tasks = pd.read_csv(CFG.input_dir / "tasks.csv")

In [12]:
tdcsfog_subject_folds = pd.read_csv(CFG.input_dir / "tdcsfog_subject_5folds.csv")
defog_subject_folds = pd.read_csv(CFG.input_dir / "defog_subject_5folds.csv")

# Preprocessing

In [13]:
import glob
data_list = glob.glob(str(CFG.input_dir / "train/defog/*.csv"))

In [14]:
num_array = []
target_array = []
id_list = []
mask_array = []
pred_use_array = []
time_array = []
valid_array = []

for i in tqdm(defog_metadata["Id"].values):
    path = CFG.input_dir / "train/defog" / f"{i}.csv"
    if str(path) not in data_list:
        continue
    df = pd.read_csv(path)

    df["valid"] = df["Valid"] & df["Task"]
    df["valid"] = df["valid"].astype(int)

    # create features
    cols = ["AccV", "AccML", "AccAP"]
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)

    # scaling
    num_cols = [
        "AccV", "AccML", "AccAP",
        "AccV_lag_diff", "AccV_lead_diff",
        "AccML_lag_diff", "AccML_lead_diff",
        "AccAP_lag_diff", "AccAP_lead_diff",
    ]
    sc = StandardScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)

    num = df[num_cols].values
    target = df[CFG.target_cols].values
    time_values = df["Time"].values
    valid = df["valid"].values

    batch = (len(df)-1) // CFG.shift
    batch = max(1, batch)
    num_array_ = np.zeros([batch, CFG.seq_len, len(num_cols)])
    target_array_ = np.zeros([batch, CFG.seq_len, len(CFG.target_cols)])
    time_array_ = np.zeros([batch, CFG.seq_len], dtype=int)
    mask_array_ = np.zeros([batch, CFG.seq_len], dtype=int)
    pred_use_array_ = np.zeros([batch, CFG.seq_len], dtype=int)
    valid_array_ = np.zeros([batch, CFG.seq_len], dtype=int)

    if len(df) <= CFG.seq_len:
        num_array_[0, :len(num), :] = num
        target_array_[0, :len(target), :] = target
        mask_array_[0, :len(target)] = 1
        pred_use_array_[0, :len(target)] = 1
        time_array_[0, :len(time_values)] = time_values
        valid_array_[0, :len(target)] = valid
        continue

    for n, b in enumerate(range(batch)):
        if b == (batch - 1):
            # get values
            num_ = num[b*CFG.shift:]
            target_ = target[b*CFG.shift:]
            time_ = time_values[b*CFG.shift:]
            valid_ = valid[b*CFG.shift:]
            # insert values
            num_array_[b, :len(num_), :] = num_
            target_array_[b, :len(target_), :] = target_
            mask_array_[b, :len(target_)] = 1
            pred_use_array_[b, CFG.offset:len(target_)] = 1
            time_array_[b, :len(time_)] = time_
            valid_array_[b, :len(valid_)] = valid_
        elif b == 0:
            # get values
            num_ = num[0:CFG.seq_len]
            target_ = target[0:CFG.seq_len]
            time_ = time_values[0:CFG.seq_len]
            valid_ = valid[0:CFG.seq_len]
            # insert values
            num_array_[0, :, :] = num_
            target_array_[0, :, :] = target_
            mask_array_[0, :] = 1
            pred_use_array_[0, :CFG.seq_len-CFG.offset] = 1
            time_array_[0, :] = time_
            valid_array_[0, :] = valid_
        else:
            # get values
            num_ = num[b*CFG.shift:b*CFG.shift+CFG.seq_len]
            target_ = target[b*CFG.shift:b*CFG.shift+CFG.seq_len]
            time_ = time_values[b*CFG.shift:b*CFG.shift+CFG.seq_len]
            valid_ = valid[b*CFG.shift:b*CFG.shift+CFG.seq_len]
            # insert values
            num_array_[b, :, :] = num_
            target_array_[b, :, :] = target_
            mask_array_[b, :] = 1
            pred_use_array_[b, CFG.offset:CFG.seq_len-CFG.offset] = 1
            time_array_[b, :] = time_
            valid_array_[b, :] = valid_

    num_array.append(num_array_)
    target_array.append(target_array_)
    mask_array.append(mask_array_)
    pred_use_array.append(pred_use_array_)
    time_array.append(time_array_)
    valid_array.append(valid_array_)
    id_list += [i for _ in range(batch)]

  0%|          | 0/137 [00:00<?, ?it/s]

In [15]:
num_array = np.concatenate(num_array, axis=0)
target_array = np.concatenate(target_array, axis=0)
mask_array = np.concatenate(mask_array, axis=0)
pred_use_array = np.concatenate(pred_use_array, axis=0)
time_array = np.concatenate(time_array, axis=0)
valid_array = np.concatenate(valid_array,axis=0)

In [16]:
df_id = pd.DataFrame()
df_id["Id"] = id_list
df_id = pd.merge(df_id, defog_metadata[["Id", "Subject"]], on="Id", how="left")
print(df_id.shape)
display(df_id.head())

(5369, 2)


,Id,Subject
0,02ea782681,ae2d35
1,02ea782681,ae2d35
2,02ea782681,ae2d35
3,02ea782681,ae2d35
4,02ea782681,ae2d35


In [17]:
num_array.shape

(5369, 5000, 9)

# CV split

In [18]:
df_id = pd.merge(df_id, defog_subject_folds, on="Subject", how="left")
print(df_id.shape)
display(df_id.head())

(5369, 3)


,Id,Subject,fold
0,02ea782681,ae2d35,0
1,02ea782681,ae2d35,0
2,02ea782681,ae2d35,0
3,02ea782681,ae2d35,0
4,02ea782681,ae2d35,0


# Dataset

In [19]:
def preprocess(numerical_array, mask_array, valid_array):
    attention_mask = mask_array == 0
    return {
        "input_data_numerical_array": numerical_array,
        "input_data_mask_array": mask_array,
        "input_data_valid_array": valid_array,
        "attention_mask": attention_mask,
    }

In [20]:
class FogDataset(Dataset):
    def __init__(
        self,
        numerical_array,
        mask_array,
        valid_array,
        train=True,
        y=None,
    ):
        self.numerical_array = numerical_array
        self.mask_array = mask_array
        self.valid_array = valid_array
        self.train = train
        self.y = y

    def __len__(self):
        return len(self.numerical_array)

    def __getitem__(self, item):
        data = preprocess(
            self.numerical_array[item],
            self.mask_array[item],
            self.valid_array[item],

        )

        # Return the processed data where the lists are converted to `torch.tensor`s
        if self.train:
            return {
              "input_data_numerical_array": torch.tensor(data["input_data_numerical_array"], dtype=torch.float32),
              "input_data_mask_array": torch.tensor(data["input_data_mask_array"], dtype=torch.long),
              "input_data_valid_array": torch.tensor(data["input_data_valid_array"], dtype=torch.long),
              "attention_mask": torch.tensor(data["attention_mask"], dtype=torch.bool),
              "y": torch.tensor(self.y[item], dtype=torch.float32),
            }
        else:
            return {
              "input_data_numerical_array": torch.tensor(data["input_data_numerical_array"], dtype=torch.float32),
              "input_data_mask_array": torch.tensor(data["input_data_mask_array"], dtype=torch.long),
              "input_data_valid_array": torch.tensor(data["input_data_valid_array"], dtype=torch.long),
              "attention_mask": torch.tensor(data["attention_mask"], dtype=torch.bool),
            }

# Model

In [21]:
class FogLstmModel(nn.Module):
    def __init__(
        self,
        dropout=0.2,
        input_numerical_size=9,
        numeraical_linear_size=64,
        model_size=128,
        linear_out=128,
        out_size=3,
    ):
        super(FogLstmModel, self).__init__()
        self.numerical_linear = nn.Sequential(
                nn.Linear(input_numerical_size, numeraical_linear_size),
                nn.LayerNorm(numeraical_linear_size)
            )
        self.lstm = nn.GRU(
            numeraical_linear_size,
            model_size,
            num_layers=2,
            batch_first=True,
            bidirectional=True,
        )
        self.linear_out = nn.Sequential(
            nn.Linear(model_size*2, linear_out),
            nn.LayerNorm(linear_out),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(linear_out, out_size),
        )
        self._reinitialize()

    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'lstm' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)

    def forward(self, numerical_array, mask_array, attention_mask):
        numerical_embedding = self.numerical_linear(numerical_array)
        output, _ = self.lstm(numerical_embedding)
        output = self.linear_out(output)
        return output

# Training

In [22]:
def train_fn(
    train_loader,
    model,
    criterion,
    optimizer,
    epoch,
    scheduler,
    device,
):
    model.train()
    losses = AverageMeter()
    start = time.time()
    for step, d in enumerate(train_loader):
        input_data_numerical_array = d["input_data_numerical_array"].to(device)
        input_data_mask_array = d["input_data_mask_array"].to(device)
        input_data_valid_array = d["input_data_valid_array"].to(device)
        attention_mask = d["attention_mask"].to(device)
        y = d["y"].to(device)
        batch_size = y.size(0)
        optimizer.zero_grad()

        output = model(input_data_numerical_array, input_data_mask_array, attention_mask)
        loss = criterion(output[input_data_valid_array==1], y[input_data_valid_array==1])
        losses.update(loss.item(), batch_size)

        loss.backward()
        optimizer.step()
        scheduler.step()

        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print(
                "Epoch: [{0}][{1}/{2}] "
                "Elapsed {remain:s} "
                "Loss: {loss.val:.4f}({loss.avg:.4f}) "
                "LR: {lr:.6f}  "
                .format(
                    epoch+1,
                    step,
                    len(train_loader),
                    remain=timeSince(start, float(step+1) / len(train_loader)),
                    loss=losses,
                    lr=scheduler.get_lr()[0],
                )
            )
    return losses.avg

In [23]:
def valid_fn(
    val_loader,
    model,
    criterion,
    device,
):
    model.eval()
    preds = []
    for d in tqdm(val_loader):
        input_data_numerical_array = d["input_data_numerical_array"].to(device)
        input_data_mask_array = d["input_data_mask_array"].to(device)
        input_data_valid_array = d["input_data_valid_array"].to(device)
        attention_mask = d["attention_mask"].to(device)

        with torch.no_grad():
            outputs = model(input_data_numerical_array, input_data_mask_array, attention_mask)

        preds.append(outputs.sigmoid().detach().cpu().numpy())

    preds = np.concatenate(preds)

    return preds

In [24]:
def train_loop(train_idx, valid_idx, i_fold, device):
    print(f"========== fold {i_fold} training ==========")

    train_numerical_array = num_array[train_idx]
    train_target_array = target_array[train_idx]
    train_mask_array = mask_array[train_idx]
    train_valid_array = valid_array[train_idx]

    val_numerical_array = num_array[valid_idx]
    val_target_array = target_array[valid_idx]
    val_mask_array = mask_array[valid_idx]
    val_pred_array = pred_use_array[valid_idx]
    val_valid_array = valid_array[valid_idx]

    train_dataset = FogDataset(
        train_numerical_array,
        train_mask_array,
        train_valid_array,
        train=True,
        y=train_target_array,
    )
    val_dataset = FogDataset(
        val_numerical_array,
        val_mask_array,
        val_valid_array,
        train=True,
        y=val_target_array,
    )
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=CFG.batch_size,
        shuffle=True,
        num_workers=CFG.num_workers,
    )
    val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=CFG.batch_size,
        shuffle=False,
        num_workers=CFG.num_workers,
    )
    print(f"Train Size: {len(train_dataset)}")
    print(f"Valid Size: {len(val_dataset)}")

    model = FogLstmModel()
    model = model.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {"params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], "weight_decay": CFG.weight_decay},
        {"params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
    ]
    optimizer = AdamW(
        optimizer_grouped_parameters,
        lr=CFG.lr,
        weight_decay=CFG.weight_decay,
    )

    num_train_optimization_steps = int(len(train_loader) * CFG.epochs)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=CFG.num_warmup_steps,
        num_training_steps=num_train_optimization_steps,
    )
    criterion = nn.BCEWithLogitsLoss()
    best_score = 0.0

    for epoch in range(CFG.epochs):
        start_time = time.time()

        avg_loss = train_fn(
            train_loader,
            model,
            criterion,
            optimizer,
            epoch,
            scheduler,
            device,
        )

        val_preds = valid_fn(
            val_loader,
            model,
            criterion,
            device,
        )

        # scoring
        pred_valid_index = (val_mask_array == 1) & (val_pred_array == 1) & (val_valid_array == 1)
        scores = [
            average_precision_score(
                val_target_array[pred_valid_index][:, i],
                val_preds[pred_valid_index][:, i],
            )
            for i in range(3)
        ]
        mean_score = np.nanmean(scores)

        elapsed = time.time() - start_time
        print(f"Fold {i_fold} Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  time: {elapsed:.0f}s")
        print(f"Fold {i_fold} Epoch {epoch+1} - Score: {mean_score:.4f}")
        print(f"Fold {i_fold} Epoch {epoch+1} - ClassWise Score: {[score for score in scores]}")

        if best_score <= mean_score:
            print(f"Fold {i_fold} Epoch {epoch+1} - Save Best Score: {best_score} -> {mean_score:.4f}")
            best_score = mean_score
            torch.save({
                "model": model.state_dict(),
                "predictions": val_preds,
                },
                CFG.output_dir / f"fold{i_fold}_best.pth",
            )

    val_preds = torch.load(
        CFG.output_dir / f"fold{i_fold}_best.pth",
        map_location=torch.device("cpu"),
    )["predictions"]

    torch.cuda.empty_cache()
    gc.collect()

    return val_preds

In [28]:
def train_all(train_idx, i_fold, device):
    print(f"========== fold {i_fold} training ==========")

    train_numerical_array = num_array[train_idx]
    train_target_array = target_array[train_idx]
    train_mask_array = mask_array[train_idx]
    train_valid_array = valid_array[train_idx]

    train_dataset = FogDataset(
        train_numerical_array,
        train_mask_array,
        train_valid_array,
        train=True,
        y=train_target_array,
    )
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=CFG.batch_size,
        shuffle=True,
        num_workers=CFG.num_workers,
    )
    print(f"Train Size: {len(train_dataset)}")

    model = FogLstmModel()
    model = model.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {"params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], "weight_decay": CFG.weight_decay},
        {"params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
    ]
    optimizer = AdamW(
        optimizer_grouped_parameters,
        lr=CFG.lr,
        weight_decay=CFG.weight_decay,
    )

    num_train_optimization_steps = int(len(train_loader) * CFG.epochs)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=CFG.num_warmup_steps,
        num_training_steps=num_train_optimization_steps,
    )
    criterion = nn.BCEWithLogitsLoss()

    for epoch in range(CFG.epochs):
        start_time = time.time()

        avg_loss = train_fn(
            train_loader,
            model,
            criterion,
            optimizer,
            epoch,
            scheduler,
            device,
        )

        elapsed = time.time() - start_time
        print(f"Fold {i_fold} Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  time: {elapsed:.0f}s")
        torch.save({
            "model": model.state_dict(),
            },
            CFG.output_dir / f"fold{i_fold}_best.pth",
        )

    torch.cuda.empty_cache()
    gc.collect()

In [29]:
def main():
    # ====================================================
    # Training
    # ====================================================
    y_oof = np.empty([len(target_array), CFG.seq_len, 3])
    if CFG.train_by_fold:
        for i_fold in range(CFG.n_fold):
            if i_fold in CFG.train_fold:
                train_idx = list(df_id.query("fold != @i_fold").index)
                valid_idx = list(df_id.query("fold == @i_fold").index)
                best_val_preds = train_loop(train_idx, valid_idx, i_fold, DEVICE)
                y_oof[valid_idx] = best_val_preds
                np.save(CFG.output_dir / f"fold{i_fold}_val_preds_{CFG.seq_len}.npy", best_val_preds)
        np.save(CFG.output_dir / f"oof_{CFG.seq_len}.npy", y_oof)

    if CFG.train_all:
        for i_fold in range(CFG.n_fold):
            train_idx = list(df_id.index)
            train_all(train_idx, i_fold, DEVICE)

In [30]:
if __name__ == "__main__":
    main()

========== fold 0 training ==========
Train Size: 5369
Epoch: [1][0/224] Elapsed 0m 1s (remain 7m 20s) Loss: 0.8165(0.8165) LR: 0.000100  
Epoch: [1][50/224] Elapsed 0m 28s (remain 1m 37s) Loss: 0.1982(0.2496) LR: 0.000983  
Epoch: [1][100/224] Elapsed 0m 55s (remain 1m 7s) Loss: 0.1491(0.2032) LR: 0.000963  
Epoch: [1][150/224] Elapsed 1m 23s (remain 0m 40s) Loss: 0.0831(0.1882) LR: 0.000943  
Epoch: [1][200/224] Elapsed 1m 50s (remain 0m 12s) Loss: 0.1181(0.1752) LR: 0.000922  
Epoch: [1][223/224] Elapsed 2m 2s (remain 0m 0s) Loss: 0.2322(0.1726) LR: 0.000913  
Fold 0 Epoch 1 - avg_train_loss: 0.1726  time: 123s
Epoch: [2][0/224] Elapsed 0m 1s (remain 4m 47s) Loss: 0.1992(0.1992) LR: 0.000912  
Epoch: [2][50/224] Elapsed 0m 28s (remain 1m 36s) Loss: 0.2042(0.1294) LR: 0.000892  
Epoch: [2][100/224] Elapsed 0m 55s (remain 1m 7s) Loss: 0.1133(0.1341) LR: 0.000872  
Epoch: [2][150/224] Elapsed 1m 24s (remain 0m 40s) Loss: 0.1302(0.1308) LR: 0.000851  
Epoch: [2][200/224] Elapsed 1m 51s 

In [31]:
import json
dataset_metadata_json = {
    "title": CFG.exp_name,
    "id": f"shuheigoda/{CFG.exp_name}",
    "licenses": [{"name": "CC0-1.0"}],
}
with open(CFG.output_dir / "dataset-metadata.json", "w") as f:
    json.dump(dataset_metadata_json, f, indent=4)
t = str(CFG.output_dir)
print(t)
!kaggle datasets create -p $t -r zip

../mnt/output/pd-exp222
Starting upload for file fold0_best.pth
100%|███████████████████████████████████████| 1.83M/1.83M [00:04<00:00, 479kB/s]
Upload successful: fold0_best.pth (2MB)
Starting upload for file fold1_best.pth
100%|███████████████████████████████████████| 1.83M/1.83M [00:02<00:00, 693kB/s]
Upload successful: fold1_best.pth (2MB)
Starting upload for file fold2_best.pth
100%|███████████████████████████████████████| 1.83M/1.83M [00:02<00:00, 729kB/s]
Upload successful: fold2_best.pth (2MB)
Starting upload for file fold3_best.pth
100%|███████████████████████████████████████| 1.83M/1.83M [00:02<00:00, 658kB/s]
Upload successful: fold3_best.pth (2MB)
Starting upload for file fold4_best.pth
100%|███████████████████████████████████████| 1.83M/1.83M [00:03<00:00, 545kB/s]
Upload successful: fold4_best.pth (2MB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/shuheigoda/pd-exp222


In [ ]:
if CFG.env == "colab":
    from google.colab import runtime
    runtime.unassign()